In [4]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.typing import NDArray
import scipy 

In [5]:
data=np.load('C:\\Users\\11203\\pyStudy\\TranFraud\\pgm\\CAP-5771-s24-hw6-main\\CAP-5771-s24-hw6-main\\question2_cluster_data.npy')
labels=np.load('C:\\Users\\11203\\pyStudy\\TranFraud\\pgm\\CAP-5771-s24-hw6-main\\CAP-5771-s24-hw6-main\\question2_cluster_labels.npy')
print(data.shape)
print(labels.shape)
n_points = data.shape[0]
indices = np.random.choice(n_points, size=5000, replace=False)

selected_data = data[indices]
selected_labels = labels[indices]
print(selected_data.shape)
print(selected_labels.shape)
list_i=[0,1,2,3,4]
slice={}
slice_labels={}
for i in list_i:
    slice[i]=selected_data[i*1000:(i+1)*1000]
    slice_labels[i]=selected_labels[i*1000:(i+1)*1000]
print(slice[0].shape)
print(slice_labels[0].shape)

(50000, 2)
(50000,)
(5000, 2)
(5000,)
(1000, 2)
(1000,)


In [16]:
def em_algorithm(data, max_iter):
   
    # 初始化参数
    weights = np.random.rand(2)
    weights /= np.sum(weights)
    means = np.random.rand(2, 2)
    covariances = np.random.rand(2, 2, 2)
    log_likelihoods = []

    # EM算法迭代过程
    for _ in range(max_iter):
        # E步：计算每个点属于每个分布的概率
        weights_norm = weights / np.sum(weights)
        prob = np.zeros((len(data), 2))
        for i in range(2):
            diff = data - means[i]
            cov_inv = np.linalg.inv(covariances[i])
            exponent = -0.5 * np.sum(diff @ cov_inv * diff, axis=1)
            coef = 1 / (2 * np.pi * np.sqrt(np.linalg.det(covariances[i])))
            prob[:, i] = coef * np.exp(exponent)
        gamma = prob * weights_norm / np.sum(prob * weights_norm, axis=1)[:, None]

        # M步：更新参数
        Nk = np.sum(gamma, axis=0)
        weights = Nk / len(data)
        means = (gamma.T @ data) / Nk[:, None]
        covariances = np.zeros((2, 2, 2))
        for i in range(2):
            diff = data - means[i]
            covariances[i] = (gamma[:, i, None] * diff).T @ diff / Nk[i]

        # 计算对数似然值并存储
        log_likelihood = np.sum(np.log(np.sum(prob * weights_norm, axis=1)))
        log_likelihoods.append(log_likelihood)

    return weights, means, covariances, log_likelihoods

In [8]:
em_weights,em_means,em_covariances,em_log_likelihoods =em_algorithm(slice[0], max_iter=100)
print(em_weights.shape)
print(em_means.shape)
print(em_covariances.shape)

(2,)
(2, 2)
(2, 2, 2)


In [15]:
def run_em_algorithm(data, num_runs):
    
    weights_sum = np.zeros(2)
    means_sum = np.zeros((2, 2))
    covariances_sum = np.zeros((2, 2, 2))
    log_likelihoods_sum = np.zeros(100)

    for _ in range(num_runs):
        weights, means, covariances, log_likelihoods = em_algorithm(data)
        weights_sum += weights
        means_sum += means
        covariances_sum += covariances
        log_likelihoods_sum += log_likelihoods

    weights_avg = weights_sum / num_runs
    means_avg = means_sum / num_runs
    covariances_avg = covariances_sum / num_runs
    log_likelihoods_avg = log_likelihoods_sum / num_runs

    return {
        "weights_avg": weights_avg,
        "means_avg": means_avg,
        "covariances_avg": covariances_avg,
        "log_likelihoods_avg": log_likelihoods_avg
    }

In [14]:
results=run_em_algorithm(slice[0],num_runs=10)
results

C:\Users\11203\AppData\Local\Temp\ipykernel_16604\1621800549.py:31: RuntimeWarning: invalid value encountered in sqrt
  coef = 1 / (2 * np.pi * np.sqrt(np.linalg.det(covariances[i])))
C:\Users\11203\AppData\Local\Temp\ipykernel_16604\1621800549.py:32: RuntimeWarning: overflow encountered in exp
  prob[:, i] = coef * np.exp(exponent)


{'weights_avg': array([nan, nan]),
 'means_avg': array([[nan, nan],
        [nan, nan]]),
 'covariances_avg': array([[[nan, nan],
         [nan, nan]],
 
        [[nan, nan],
         [nan, nan]]]),
 'log_likelihoods_avg': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan])}